# Server Activity Analysis

This is a study of Iraq BTW's activity data over a period of time.

This version (Published Sep 5) details around half a day of activity.

In [ ]:
from pymongo import MongoClient
import pymongo
from matplotlib import pyplot as plt
import matplotlib.dates as md

import os, dotenv, datetime, pytz

dotenv.load_dotenv()
mongo_client = MongoClient(os.environ.get('MONGOKEY'))
DB = mongo_client[mongo_client.list_database_names()[0]]
print(f"MongoDB Atlas Connected to Database: {mongo_client.list_database_names()[0]}")

## Storage

Using MongoDB, we store the activity logs, produced by the bot, which includes:

* Number of people online
* Number of messages sent over time
* Number of messages sent per person


Therefore, we are to retrieve this information

## Data Pull
Pull collection data from MongoDB; **do this when you want to refresh!**

In [ ]:
collection = DB["activitydata"]
entries = collection.find({})

print(f"Number of entries: {len(list(entries))}")

times = []
onlinepeople = []
messagevolume = []
cumulativemsgs = [0]

bars = {}

tz = pytz.timezone("US/Eastern")

for t in list(collection.find({})):
    times.append(md.date2num(t["time"]))
    onlinepeople.append(t["online-members"])
    messagevolume.append(t["messages-sent"])
    cumulativemsgs.append(cumulativemsgs[-1] + t["messages-sent"])

    for person in t["quantities"]:
        try: bars[person] += t["quantities"][person]
        except: bars[person] = t["quantities"][person]

print(f"Last updated: {datetime.datetime.now(tz=tz)}")


## Data Processing

Sift through the database and locate all of the time entries

## Plotting

Using `matplotlib` to produce a time graph

## Online Members

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)

ax.set_title("Number of Online Members on Iraq BTW")
ax.plot_date(times, onlinepeople, "-")

ax.set_ylabel("Number of Members (Human, Online)")
ax.set_xlabel("Time")

xfmt = md.DateFormatter('%H:%M', tz=tz)
ax.xaxis.set_major_formatter(xfmt)

ax.grid()



## Message Volume
--> Number of messages sent in the last 10 minutes

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)

ax.set_title("Message Volume")
ax.plot_date(times, messagevolume, "-")

ax.set_ylabel("Number of Messages Sent in Time Window")
ax.set_xlabel("Time")

xfmt = md.DateFormatter('%H:%M', tz=tz)
ax.xaxis.set_major_formatter(xfmt)


ax.grid()

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)

ax.set_title("Cumulative Messages")
ax.plot_date(times, cumulativemsgs[1:], "-")

ax.set_ylabel("Cumulative Number of Messages")
ax.set_xlabel("Time")

xfmt = md.DateFormatter('%H:%M', tz=tz)
ax.xaxis.set_major_formatter(xfmt)


ax.grid()

## Instantaneous Rate of messaging

We can use SciPy's Spline interpolator to get an approximate measure of the instantaneous rate of messaging

In [ ]:
from scipy.interpolate import UnivariateSpline
import numpy as np

x = times
y = cumulativemsgs[1:]

fc = UnivariateSpline(x, y)

xs = np.linspace(x[0], x[-1], num=100)

fig = plt.figure()

ax = fig.add_subplot(111)

ax.set_title("Approx. Cumulative Messages")
ax.plot_date(xs, fc(xs), "-")

ax.set_ylabel("Number of Messages")
ax.set_xlabel("Time")

xfmt = md.DateFormatter('%H:%M', tz=tz)
ax.xaxis.set_major_formatter(xfmt)


ax.grid()

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)

ax.set_title("Approx. Rate of Messages")
ax.plot_date(xs, fc.derivative()(xs)/1440, "-")
# Need to divide by 1440 -- number of minutes per day

ax.set_ylabel("Messages per Minute")
ax.set_xlabel("Time")

xfmt = md.DateFormatter('%H:%M', tz=tz)
ax.xaxis.set_major_formatter(xfmt)


ax.grid()

## Messages by person

Using discord.py we can take the names of the individuals from their id and produce plots

In [ ]:
import pickle

with open("names", "rb") as f:
    names = pickle.load(f)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)

xs = np.arange(len(bars))

sorted_keys = [e for _, e in sorted(zip(bars.values(), bars.keys()))]

ax.set_title("Number of Messages by Person")
ax.barh(xs, sorted(bars.values()))

ax.set_yticks(np.arange(len(bars)))
ax.set_yticklabels([names[int(i)] for i in sorted_keys])

plt.show()